<a href="https://colab.research.google.com/github/jsilveira1409/CIVIL-459-Animal-Pose-Estimation/blob/sda/openpifpaf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
!pip3 install openpifpaf

import openpifpaf
import IPython
from openpifpaf.transforms import Crop
from PIL import Image
import gdown
import subprocess
openpifpaf.show.Canvas.show = True


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: C:\Users\osour\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
!git clone --branch sda https://github.com/jsilveira1409/CIVIL-459-Animal-Pose-Estimation.git plugins
!mv plugins/openpifpaf_animalplugin/ openpifpaf_animalplugin
from openpifpaf_animalplugin import AnimalPoseEstimation

Cloning into 'plugins'...


In [13]:
!rm -rf data-animalpose/
!mkdir data-animalpose
!gdown "https://drive.google.com/drive/folders/1xxm6ZjfsDSmv6C9JvbgiGrmHktrUjV5x" -O data-animalpose --folder 
#!mv animalpose_keypoint_new/ data-animalpose/
!unzip data-animalpose/images.zip -d data-animalpose/
!rm data-animalpose/images.zip

usage: gdown [-h] [-V] [-O OUTPUT] [-q] [--fuzzy] [--id] [--proxy PROXY]
             [--speed SPEED] [--no-cookies] [--no-check-certificate]
             [--continue] [--folder] [--remaining-ok] [--format FORMAT]
             url_or_id
gdown: error: unrecognized arguments: data-animalpose
unzip:  cannot find or open data-animalpose/images.zip, data-animalpose/images.zip.zip or data-animalpose/images.zip.ZIP.
rm: cannot remove 'data-animalpose/images.zip': No such file or directory


In [5]:
config = openpifpaf.plugin.register()
openpifpaf.DATAMODULES

{'animal': openpifpaf.plugins.animalpose.animal_kp.AnimalKp,
 'apollo': openpifpaf.plugins.apollocar3d.apollo_kp.ApolloKp,
 'cifar10': openpifpaf.plugins.cifar10.datamodule.Cifar10,
 'cocodet': openpifpaf.plugins.coco.cocodet.CocoDet,
 'cocokp': openpifpaf.plugins.coco.cocokp.CocoKp,
 'crowdpose': openpifpaf.plugins.crowdpose.module.CrowdPose,
 'nuscenes': openpifpaf.plugins.nuscenes.nuscenes.NuScenes,
 'posetrack2018': openpifpaf.plugins.posetrack.posetrack2018.Posetrack2018,
 'posetrack2017': openpifpaf.plugins.posetrack.posetrack2017.Posetrack2017,
 'cocokpst': openpifpaf.plugins.posetrack.cocokpst.CocoKpSt,
 'wholebody': openpifpaf.plugins.wholebody.wholebody.Wholebody,
 'custom_animal': openpifpaf_animalplugin.dataset.AnimalPoseEstimation}

In [10]:
# create a dataset
dataset = AnimalPoseEstimation(image_dir='data-animalpose/images', annotation_file='data-animalpose/keypoints.json')

FileNotFoundError: [Errno 2] No such file or directory: 'data-animalpose/keypoints.json'

In [11]:
!python3 -m openpifpaf.train \
    --dataset custom_animal \
    --basenet=shufflenetv2k16 \
    --lr=0.00002 \
    --momentum=0.95 \
    --epochs=300 \
    --lr-decay 160 260 \
    --lr-decay-epochs=10  \
    --weight-decay=1e-5 \
    --weight-decay=1e-5 \
    --val-interval 10 \
    --loader-workers 16 \
    --batch-size 8

INFO:__main__:neural network device: cpu (CUDA available: False, count: 0)


Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\osour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpifpaf\train.py", line 196, in <module>
    main()
  File "C:\Users\osour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpifpaf\train.py", line 128, in main
    args = cli()
  File "C:\Users\osour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\openpifpaf\train.py", line 120, in cli
   